# Veri Yogun Uygulamalar Modul Projesi: Zaman serisi siniflandirma problemi

Dr. Ismail Guzel

Milli Teknoloji Hamlesi altinda Yapay Zeka Uzmanlık Programı kapsamında 18 saatlik Veri Yogun Uygulamar egitimi sonunda bu projenin tamamlanmasi beklenilmektedir.

Bu proje icerisinde amac farkli sektorlerden elde dilmis zaman serileri uzerinden elde edilen faktorler uzerine kurulmus bir classification modeli kurarak benzerlik calismasi yapmaktir.

Mesela ilgilenilen bir hisse senedi X olsun, bunun bulundugu sektor bazli diger hisse senetlerin davranislarindan farkli davrandigini dusunelim. Yani sektor icinde bir artis gozlemlenirken bu hisse senetinde bir hareketlilik olmasin. Dolayisiyla, hangi sektore daha cok benziyor sorusuna cevap verebilirsek, o sektor'un hareketlerine gore bir hipotez kurabiliriz.

Bu proje, asagidaki surecleri kapsayacak:

- Sektorleriin listesine bir web-scraping ile erisilmesi ve verilerin elde edilmesi (`yfinance`, `investpy`, [`quandl`]((https://docs.data.nasdaq.com/v1.0/docs/python-installation)))
- 2005-01-01 yilindan itibaren aylik getirelerden olusan serilerin elde edilmesi
- 3 buyuk sektor uzerinden getirilerin faktorleri(momentum gibi) hesaplanmasi
- Bu momentum serileri uzerinden bir tsfresh ile feature engineering yapilmasi (imputing, encoding, transformation, ve daha fazlasi)
- Yeni elde edilmis feature ve sektor siniflari uzerinden bir model kurulmasi (en iyi model secmesi)
- Diger sektorlerden ornekler alip ayni feature engine yontemleri yaptik sonra hangi sektore benzedigine karar vermek.
- **Bonus** Mesela Real-Estate sektorunde bulunan butun sembollerin tahmini edildikten sonra cogunluk hangi sektore(T,F,H) benzedigi bilgisine erismek.

In [ ]:
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_sectors_names():
    url = "https://stockanalysis.com/stocks/industry/sectors/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
    else:
        print(f"Error: Failed to fetch data from page {url}")

    return df

def fetch_industry_names():
    url = "https://stockanalysis.com/stocks/industry/all/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
    else:
        print(f"Error: Failed to fetch data from page {url}")

    return df

def fetch_data(sectors):
    url = f"https://stockanalysis.com/stocks/sector/{sectors}/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
        df.drop(columns='No.', inplace=True)
    else:
        print(f"Error: Failed to fetch data from page {url}")

    return df

In [2]:
sectors=fetch_sectors_names()
industry=fetch_industry_names()

In [3]:
sectors

,Sector Name,Stocks,Market Cap,Div. Yield,PE Ratio,Profit Margin,1D Change,1Y Change
0,Financials,1384,"9,687.25B",2.41%,14.70,17.75%,0.54%,11.61%
1,Healthcare,1217,"8,168.29B",0.43%,50.93,4.07%,1.32%,7.52%
2,Technology,788,17.72T,0.43%,44.88,13.16%,0.27%,14.87%
3,Industrials,652,"5,478.40B",1.11%,26.71,7.32%,0.55%,17.56%
4,Consumer Discretionary,577,"7,264.64B",0.65%,27.33,5.95%,0.31%,2.23%
5,Real Estate,264,"1,506.14B",4.13%,50.70,8.89%,-0.12%,6.92%
6,Materials,263,"2,069.59B",1.56%,19.61,8.68%,0.46%,2.26%
7,Communication Services,260,"5,361.58B",1.09%,28.01,10.40%,0.30%,2.10%
8,Energy,253,"3,646.11B",2.85%,8.01,12.42%,0.43%,18.22%
9,Consumer Staples,241,"4,035.49B",1.45%,29.74,4.72%,0.45%,12.15%


## Sektor listerine erismek

Yukarida yazilan fonksyionlar ile hangi sembollerin hangi sektorlerde oldugu bilgisine erisim saglanabilir ve asagidaki betikler yardimi ile `.csv` dosyalarinda saklayabiliriz. Sonrasinda, sektor bazli sembollere ait verileri indirilebilir ve siniflandirilabilir.

In [6]:
!pwd

/content


In [8]:
fetch_data(sectors='energy').to_csv('data/stock_sectors/energy.csv')
fetch_data(sectors='financials').to_csv('data/stock_sectors/financials.csv')
fetch_data(sectors='healthcare').to_csv('data/stock_sectors/healthcare.csv')
fetch_data(sectors='technology').to_csv('data/stock_sectors/technology.csv')
fetch_data(sectors='utilities').to_csv('data/stock_sectors/utilities.csv')
fetch_data(sectors='real-estate').to_csv('data/stock_sectors/real-estate.csv')
fetch_data(sectors='materials').to_csv('data/stock_sectors/materials.csv')
fetch_data(sectors='technology').to_csv('data/stock_sectors/technology.csv')
fetch_data(sectors='industrials').to_csv('data/stock_sectors/industrials.csv')
fetch_data(sectors='consumer-staples').to_csv('data/stock_sectors/consumer-staples.csv')
fetch_data(sectors='consumer-discretionary').to_csv('data/stock_sectors/consumer-discretionary.csv')
fetch_data(sectors='communication-services').to_csv('data/stock_sectors/communication-services.csv')

In [9]:
finance = pd.read_csv('data/stock_sectors/financials.csv')
technology = pd.read_csv('data/stock_sectors/technology.csv')
healthcare = pd.read_csv('data/stock_sectors/healthcare.csv')

finance_symbols = finance['Symbol'].tolist()
technology_symbols = technology['Symbol'].tolist()
healthcare_symbols = healthcare['Symbol'].tolist()


## Veriye erismek
Diyelim ki, finans sektorunden `HSBC` sembolu icin verileri indirmek istiyoruz. Bu adim icin `yfinance` kullanilabilir. Oncelikle `.Ticker` ile bir object olusturup onun uzerinden dogru hissemi olduguna dair bilgileri teyit edebiliriz. Sonrasinda `.get_history_metadata()` ile sembolun metedatasina erisim saglayabiliriz. Sonrasinda, `.history(period='3y')` ile 3 yillik veriyi calisma ortamimiza indirebiliriz.

In [ ]:
import yfinance
ticker_name = yfinance.Ticker("HSBC")
ticker_name.info

{'address1': '8 Canada Square',
 'city': 'London',
 'zip': 'E14 5HQ',
 'country': 'United Kingdom',
 'phone': '44 20 7991 8888',
 'fax': '44 20 7992 4880',
 'website': 'https://www.hsbc.com',
 'industry': 'Banks - Diversified',
 'industryKey': 'banks-diversified',
 'industryDisp': 'Banks - Diversified',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'HSBC Holdings plc provides banking and financial services worldwide. The company operates through Wealth and Personal Banking, Commercial Banking, and Global Banking and Markets segments. The Wealth and Personal Banking segment offers retail banking and wealth products, including current and savings accounts, mortgages and personal loans, credit and debit cards, and local and international payment services; and wealth management services comprising insurance and investment products, global asset management services, investment management, and private wealth 

In [ ]:
ticker_name.get_history_metadata()

{'currency': 'USD',
 'symbol': 'HSBC',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 932131800,
 'regularMarketTime': 1710532802,
 'hasPrePostMarketData': True,
 'gmtoffset': -14400,
 'timezone': 'EDT',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 38.21,
 'chartPreviousClose': 37.44,
 'previousClose': 37.53,
 'scale': 3,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EDT',
   'start': 1710489600,
   'end': 1710509400,
   'gmtoffset': -14400},
  'regular': {'timezone': 'EDT',
   'start': 1710509400,
   'end': 1710532800,
   'gmtoffset': -14400},
  'post': {'timezone': 'EDT',
   'start': 1710532800,
   'end': 1710547200,
   'gmtoffset': -14400}},
 'tradingPeriods':                                           pre_start                   pre_end  \
 Date                                                                            
 2024-03-11 00:00:00-04:00 2024-03-08 04:00:00-05:00 2024-03-08 09:30:00-05:00   
 2024-03-12 00:00

In [ ]:
data=ticker_name.history(period='3y')
data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-03-07 00:00:00-05:00,37.830002,38.029999,37.709999,37.880001,2194500,1.55,0.0
2024-03-08 00:00:00-05:00,37.570000,37.720001,37.360001,37.439999,1886400,0.00,0.0
2024-03-11 00:00:00-04:00,37.029999,37.200001,36.930000,37.130001,2471600,0.00,0.0
2024-03-12 00:00:00-04:00,37.980000,38.090000,37.840000,38.009998,2409500,0.00,0.0
2024-03-13 00:00:00-04:00,38.119999,38.189999,37.790001,37.990002,2041000,0.00,0.0


Simdi ise, belirlenen sembollerin verilerini belli bir tarih sonrasinda cekebiliriz. Sonrasinda aylik getirileri hesaplayabiliriz. Proje kapsaminda bu yontem kullanilacak.

In [10]:
# Her bir sektördeki hisse senetleri için yfinance kullanarak aylık getirileri hesaplayalım.
sectors = {'Finance': finance['Symbol'].tolist(),
           'Technology': technology['Symbol'].tolist(),
           'Healthcare': healthcare['Symbol'].tolist()}
sectors

{'Finance': ['BRK.B',
  'V',
  'JPM',
  'MA',
  'BAC',
  'WFC',
  'AXP',
  'BX',
  'HSBC',
  'MS',
  'RY',
  'HDB',
  'SPGI',
  'GS',
  'SCHW',
  'PGR',
  'MUFG',
  'BLK',
  'C',
  'TD',
  'CB',
  'UBS',
  'MMC',
  'IBN',
  'KKR',
  'CME',
  'ICE',
  'SMFG',
  'SAN',
  'MCO',
  'BMO',
  'PYPL',
  'ITUB',
  'BBVA',
  'USB',
  'BN',
  'AON',
  'APO',
  'BNS',
  'PNC',
  'NU',
  'AJG',
  'MET',
  'AIG',
  'COF',
  'ING',
  'TRV',
  'MFG',
  'AFL',
  'TFC',
  'CM',
  'IBKR',
  'MFC',
  'MSCI',
  'BSBR',
  'ALL',
  'AMP',
  'ARES',
  'BK',
  'PRU',
  'LYG',
  'BCS',
  'ACGL',
  'NDAQ',
  'SLF',
  'DFS',
  'BBD',
  'DB',
  'HIG',
  'PUK',
  'WTW',
  'NWG',
  'SYM',
  'TROW',
  'RJF',
  'IX',
  'OWL',
  'RKT',
  'BRO',
  'FITB',
  'MTB',
  'TW',
  'FCNCO',
  'FCNCA',
  'WRB',
  'STT',
  'KB',
  'ERIE',
  'LPLA',
  'MKL',
  'PFG',
  'CBOE',
  'SHG',
  'HBAN',
  'CINF',
  'NMR',
  'FDS',
  'RF',
  'L',
  'EG',
  'SYF',
  'NTRS',
  'TPG',
  'CG',
  'BAM',
  'CRBG',
  'CFG',
  'RYAN',
  'BEN',
  

In [ ]:
ticker_list=['AAPL', 'NVDA', 'INTC', 'ORCL', 'SONY']

data = yfinance.download(ticker_list, start='2005-01-01')
data_close=data['Open'].resample('M').first().pct_change()+1

[*********************100%%**********************]  5 of 5 completed


In [ ]:
data_close

Ticker,AAPL,INTC,NVDA,ORCL,SONY
Date,,,,,
2005-01-31,NaN,NaN,NaN,NaN,NaN
2005-02-28,1.189410,0.951354,0.939294,0.989193,0.947125
2005-03-31,1.167813,1.083148,1.271616,0.949017,1.030894
2005-04-30,0.935541,0.958128,0.825893,0.963929,1.050210
2005-05-31,0.860299,1.006855,0.922661,0.921178,0.932416
...,...,...,...,...,...
2023-11-30,0.998715,1.024993,0.928549,0.981004,1.032819
2023-12-31,1.113041,1.215342,1.137976,1.118208,1.012213
2024-01-31,0.983292,1.109107,1.058442,0.900491,1.095232


In [ ]:
# 3 aylik momentum faktorleri
# Kumulatif getiriyi 3 ayı çarparak elde ediyoruz.

import numpy as np

def get_rolling_ret(data, n):
    return data.rolling(n).apply(np.prod)
calisilacak_veri = get_rolling_ret(data_close, 3).dropna()
calisilacak_veri

NameError: name 'data_close' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[11,8])

for ticker in ticker_list:
    plt.plot(calisilacak_veri[ticker].cumprod(), label = ticker)

plt.yscale('log')
plt.title('Total Returns')
plt.legend()
plt.show()

NameError: name 'ticker_list' is not defined

<Figure size 1100x800 with 0 Axes>

In [ ]:
# Tsfresh'i kullanarak feature engineering için bilgileri çıkartabiliriz.

In [ ]:
calisilacak_veri

NameError: name 'calisilacak_veri' is not defined

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np

finance_symbols = [symbol for symbol in finance_symbols if pd.notnull(symbol) and symbol != '']
technology_symbols = [symbol for symbol in technology_symbols if pd.notnull(symbol) and symbol != '']
healthcare_symbols = [symbol for symbol in healthcare_symbols if pd.notnull(symbol) and symbol != '']


# Sektör bazında hisse senedi sembolleri
#finance_symbols = ['JPM', 'BAC', 'HSBC']
#technology_symbols = ['AAPL', 'MSFT', 'GOOGL']
#healthcare_symbols = ['JNJ', 'PFE', 'MRK']

def get_rolling_ret(data, n):
    return data.rolling(n).apply(np.prod)

# Fonksiyonu güncelleyerek doğrudan momentum hesaplaması dahil edelim
def get_sector_data(symbols, sector_name):
    sector_data = pd.DataFrame()
    for symbol in symbols:
        print(f"Downloading {symbol}")
        data = yf.download(symbol, start='2005-01-01')
        if not data.empty:
            # Aylık açılış fiyatları üzerinden aylık getirilerin hesaplanması
            data_monthly_return = data['Open'].resample('M').first().pct_change() + 1
            # 3 aylık momentum hesaplaması
            data_momentum = get_rolling_ret(data_monthly_return, 3).dropna()
            #data = data.dropna(subset=['3M Momentum'])  # NaN değerlerin momentum sütunu için temizlenmesi
            temp_df = pd.DataFrame(data_momentum)
            temp_df.rename(columns={'Open': 'Monthly Return'}, inplace=True)
            temp_df['Sector'] = sector_name  # Her satıra sektör etiketi ekleme
            temp_df['Symbol'] = symbol  # Sembol bilgisini saklama
            sector_data = pd.concat([sector_data, temp_df])
    return sector_data.dropna()

# Her sektör için veri alma
finance_data = get_sector_data(finance_symbols, 'Finance')
technology_data = get_sector_data(technology_symbols, 'Technology')
healthcare_data = get_sector_data(healthcare_symbols, 'Healthcare')

# Tüm sektörlerden gelen verileri birleştirme
all_data = pd.concat([finance_data, technology_data, healthcare_data])

# İndeksi sıfırlama ve 'Date' sütununu koruma
all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date'}, inplace=True)



[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [64]:
# Öncelikle, 'Date' sütununu datetime formatına çevirin
all_data['Date'] = pd.to_datetime(all_data['Date'])
all_data
copy_data = all_data.copy()

In [70]:
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Comprehensive featureları çıkarıyoruz
extracted_features = extract_features(all_data, column_id='Symbol', column_sort='Date', column_value='Monthly Return',
                                     default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())



Feature Extraction: 100%|██████████| 2962/2962 [04:20<00:00, 11.37it/s]


In [71]:
# Impute işlemini gerçekleştiriyoruz
impute(extracted_features)

scaler = StandardScaler()
# Sadece sayısal özellikleri içeren DataFrame oluşturma
numerical_features = extracted_features.select_dtypes(include=[float])

# Sayısal özellikleri ölçeklendirme
scaled_features = scaler.fit_transform(numerical_features)

# Ölçeklendirilmiş özellikleri DataFrame'e dönüştürme
scaled_features_df = pd.DataFrame(scaled_features, index=numerical_features.index, columns=numerical_features.columns)

# Eğer ölçeklendirilmiş veriyi orijinal `Date` ve `Symbol` bilgileriyle birleştirmek isterseniz, bu adımı ekleyebilirsiniz
# Örneğin:
# final_df = your_dataframe[['Symbol', 'Date']].merge(scaled_features_df, left_index=True, right_index=True)

print(scaled_features_df.shape)
scaled_features_df

/usr/local/lib/python3.10/dist-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['Monthly Return__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


(2962, 783)


,Monthly Return__variance_larger_than_standard_deviation,Monthly Return__has_duplicate_max,Monthly Return__has_duplicate_min,Monthly Return__has_duplicate,Monthly Return__sum_values,Monthly Return__abs_energy,Monthly Return__mean_abs_change,Monthly Return__mean_change,Monthly Return__mean_second_derivative_central,Monthly Return__median,...,Monthly Return__fourier_entropy__bins_5,Monthly Return__fourier_entropy__bins_10,Monthly Return__fourier_entropy__bins_100,Monthly Return__permutation_entropy__dimension_3__tau_1,Monthly Return__permutation_entropy__dimension_4__tau_1,Monthly Return__permutation_entropy__dimension_5__tau_1,Monthly Return__permutation_entropy__dimension_6__tau_1,Monthly Return__permutation_entropy__dimension_7__tau_1,Monthly Return__query_similarity_count__query_None__threshold_0.0,Monthly Return__mean_n_absolute_max__number_of_maxima_7
A,-0.186924,-0.025994,-0.045053,-0.357703,0.673021,-0.024482,-0.115898,0.010413,-0.020201,0.701538,...,-0.134259,0.092300,0.910811,0.389957,0.549716,0.836521,0.946781,0.997856,0.0,-0.067517
AACI,-0.186924,-0.025994,-0.045053,-0.357703,-0.669064,-0.025015,-0.182125,0.010130,-0.018261,0.371637,...,1.048572,0.689425,-0.734268,-0.269328,-0.668572,-1.243672,-1.263823,-1.318124,0.0,-0.087824
AACT,-0.186924,-0.025994,-0.045053,-0.357703,-0.810768,-0.025068,-0.189365,0.011840,-0.016822,0.366583,...,0.798605,-0.634338,-2.880985,-5.580175,-7.059173,0.352018,0.275935,0.252281,0.0,-0.055453
AADI,-0.186924,-0.025994,-0.045053,-0.357703,-0.436050,-0.024922,0.008750,-0.000045,-0.018732,-0.952221,...,0.248766,0.767076,0.231555,0.385163,0.518873,0.152502,-0.133909,-0.199872,0.0,-0.046744
AAMC,-0.186924,-0.025994,-0.045053,-0.357703,-0.628115,-0.024922,0.199981,0.056690,-0.136371,2.450991,...,-1.340442,-1.523834,-1.534759,-1.551651,-1.916183,-1.584767,-1.406666,-1.456615,0.0,0.009166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZURA,-0.186924,-0.025994,-0.045053,-0.357703,-0.795383,-0.025067,-0.091745,0.015604,0.023108,-1.868652,...,1.633781,0.065666,-1.946323,-1.979771,-2.905555,-3.073400,-3.054532,-3.268112,0.0,-0.107298
ZVRA,-0.186924,-0.025994,-0.045053,-0.357703,-0.173928,-0.024779,0.032805,0.007731,-0.017293,-0.334711,...,1.017625,1.305829,1.117019,0.384043,0.309129,0.200595,0.226288,0.247508,0.0,-0.018995
ZVSA,-0.186924,-0.025994,-0.045053,-0.357703,-0.755248,-0.025052,-0.034733,-0.057529,-0.041598,-3.632245,...,1.446681,0.896616,-1.185368,0.115961,-0.875841,-1.287943,-1.485941,-1.534078,0.0,-0.088323
ZYME,-0.186924,-0.025994,-0.045053,-0.357703,-0.294748,-0.024850,-0.046678,0.025052,-0.024718,-0.001538,...,-0.720105,-0.480480,-0.011977,0.495322,0.500540,0.242496,0.006450,-0.055565,0.0,-0.045046


In [42]:
from sklearn.feature_selection import VarianceThreshold

# Varyans eşiğine göre özellikleri filtreleme
def filter_features_by_variance(data, threshold=0.01):
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(data)
    features = data.loc[:, selector.get_support()]
    return features

# Filtre uygulama high variance kullanarak
high_variance_features = filter_features_by_variance(extracted_features, threshold=0.01)
high_variance_features

,Monthly Return__variance_larger_than_standard_deviation,Monthly Return__has_duplicate,Monthly Return__sum_values,Monthly Return__abs_energy,Monthly Return__mean_abs_change,Monthly Return__mean_change,Monthly Return__mean_second_derivative_central,Monthly Return__median,Monthly Return__mean,Monthly Return__length,...,Monthly Return__fourier_entropy__bins_3,Monthly Return__fourier_entropy__bins_5,Monthly Return__fourier_entropy__bins_10,Monthly Return__fourier_entropy__bins_100,Monthly Return__permutation_entropy__dimension_3__tau_1,Monthly Return__permutation_entropy__dimension_4__tau_1,Monthly Return__permutation_entropy__dimension_5__tau_1,Monthly Return__permutation_entropy__dimension_6__tau_1,Monthly Return__permutation_entropy__dimension_7__tau_1,Monthly Return__mean_n_absolute_max__number_of_maxima_7
A,0.0,0.0,236.925791,251.159035,0.089759,0.000646,-0.000456,1.040360,1.039148,228.0,...,0.371471,0.608606,0.960265,2.676275,1.725531,2.940639,4.090186,4.927242,5.281674,1.348095
AACI,0.0,0.0,26.314310,26.640715,0.013338,0.000496,0.000313,1.008284,1.012089,26.0,...,0.796312,0.895333,1.475076,2.304619,1.697574,2.630555,2.902002,2.978508,2.995732,1.033120
AACT,0.0,0.0,7.094596,7.190584,0.004072,0.001092,-0.000889,1.013685,1.013514,7.0,...,0.562335,1.039721,1.386294,1.386294,1.332179,1.386294,1.098612,0.693147,-0.000000,1.526853
AADI,0.0,0.0,65.599509,70.196430,0.233727,-0.005420,-0.000019,0.846825,0.923937,71.0,...,0.411314,0.847582,1.355240,2.868818,1.729351,2.853038,3.681942,4.105637,4.174387,1.771827
AAMC,0.0,0.0,28.757869,54.298800,0.467492,0.032103,-0.004321,1.145719,1.307176,22.0,...,0.721464,0.836988,0.836988,1.698783,1.330824,1.867880,2.274241,2.588573,2.685945,2.372135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZURA,0.0,0.0,7.750141,6.323692,0.172833,0.003682,0.017228,0.715047,0.775014,10.0,...,1.011404,1.242453,1.242453,1.791759,1.732868,1.945910,1.791759,1.609438,1.386294,0.839173
ZVRA,0.0,0.0,103.659653,123.326305,0.256257,-0.001109,0.000718,0.923679,0.987235,105.0,...,0.509223,0.704733,1.316201,3.085565,1.733369,2.928497,3.927882,4.361741,4.553111,2.029818
ZVSA,0.0,0.0,13.476977,11.034247,0.188933,-0.023136,0.008307,0.501171,0.585956,23.0,...,0.959615,1.357978,1.791759,2.369382,1.657700,2.441215,2.725550,2.890372,2.833213,1.026224
ZYME,0.0,0.0,84.728279,96.883221,0.195538,0.009319,-0.000635,1.004878,1.046028,81.0,...,0.452869,0.751300,1.213310,2.402211,1.752577,2.992655,3.865629,4.257770,4.317488,1.690687


In [25]:
# `all_data` DataFrame'inde her sembol için ilk tarihi ve bu tarihe ait sektör etiketini alın
grouped = all_data.groupby('Symbol').first().reset_index()


In [26]:
grouped

,Symbol,Date,Monthly Return,Sector
0,A,2005-04-30,0.929461,Healthcare
1,AACI,2022-02-28,1.006141,Finance
2,AACT,2023-09-30,1.008876,Finance
3,AADI,2018-05-31,0.750538,Healthcare
4,AAMC,2022-06-30,0.611545,Finance
...,...,...,...,...
2957,ZURA,2023-06-30,0.717884,Healthcare
2958,ZVRA,2015-07-31,1.545447,Healthcare
2959,ZVSA,2022-05-31,1.010152,Healthcare
2960,ZYME,2017-07-31,0.622222,Healthcare


In [45]:
from tsfresh import select_features
from sklearn.preprocessing import LabelEncoder

# Hedef değişkeni (Sector) sayısal formata dönüştürme
le = LabelEncoder()
y = le.fit_transform(grouped['Sector'])

# 'Symbol' sütununu kullanarak özelliklerin sıralamasını gruplandırılmış DataFrame ile eşleştirin
features_ordered = high_variance_features.loc[grouped['Symbol']]

# Şimdi features_ordered ve y, eşit sayıda satıra sahip olmalı ve select_features fonksiyonu çalıştırılabilir
relevant_features = select_features(features_ordered, y)

In [57]:
from sklearn.decomposition import PCA

# Sadece sayısal özellikleri içeren DataFrame oluşturma
numeric_features = relevant_features.select_dtypes(include=[float])


pca = PCA() # Açıklanan varyansın %95'ini koruyarak PCA uygulanıyor
pca_features = pca.fit_transform(numeric_features)
print(pca_features.shape)

(2962, 516)


In [58]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Özellikler ve hedef değişken
X = pca_features
y = le.fit_transform(grouped['Sector'])

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train



array([[-8.37722331e+03, -1.16215093e+03,  3.00670569e+02, ...,
         1.30276100e-11,  2.16674833e-12,  1.25678489e-13],
       [-8.36059349e+03, -1.16270831e+03,  3.00037201e+02, ...,
         1.08394130e-11, -1.07168689e-11, -3.90591431e-12],
       [-8.58010660e+03, -1.15621345e+03,  6.78026313e+02, ...,
        -7.23047833e-13, -1.11138435e-11,  4.02802756e-12],
       ...,
       [-8.56694740e+03, -1.15675571e+03,  3.00963796e+02, ...,
         1.24446699e-11, -1.84669967e-11, -9.74003755e-12],
       [-8.37091859e+03, -1.16251751e+03,  3.07024214e+02, ...,
         1.35264765e-11, -1.05207808e-11,  5.22357242e-13],
       [-8.37342181e+03, -1.16240396e+03,  3.03681427e+02, ...,
         2.46010616e-13,  7.95617341e-12,  7.83639113e-12]])

In [59]:
# Modeli eğitme
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Modeli değerlendirme
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Model Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.70      0.74      0.72       191
           1       0.57      0.88      0.69       228
           2       0.64      0.16      0.25       174

    accuracy                           0.62       593
   macro avg       0.64      0.59      0.56       593
weighted avg       0.63      0.62      0.57       593

Model Accuracy: 0.6222596964586846
              precision    recall  f1-score   support

           0       0.70      0.74      0.72       191
           1       0.57      0.88      0.69       228
           2       0.64      0.16      0.25       174

    accuracy                           0.62       593
   macro avg       0.64      0.59      0.56       593
weighted avg       0.63      0.62      0.57       593



In [60]:
# Hyperparameter tuning with GridSearchCV
parameters = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
clf = GridSearchCV(model, parameters, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# Evaluate the best model found by GridSearchCV
best_model = clf.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Best model's accuracy on the test set:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Best model's accuracy on the test set: 0.6290050590219224
              precision    recall  f1-score   support

           0       0.72      0.76      0.74       191
           1       0.57      0.88      0.69       228
           2       0.65      0.16      0.26       174

    accuracy                           0.63       593
   macro avg       0.65      0.60      0.56       593
weighted avg       0.64      0.63      0.58       593



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Confusion matrix oluşturma
cm = confusion_matrix(y_test, y_pred_best)

# Confusion matrix'i görselleştirme
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

# Sınıfların isimlerini eksenlere ekleyin (Varsa)
class_names = le.inverse_transform([0, 1, 2])  # Sınıf isimlerini dönüştürme
plt.xticks(ticks=[0.5, 1.5, 2.5], labels=class_names)
plt.yticks(ticks=[0.5, 1.5, 2.5], labels=class_names, rotation=0)

plt.show()



NameError: name 'y_pred_best' is not defined

In [ ]:
real_estate = pd.read_csv('../data/stock_sectors/real-estate.csv')

real_estate_symbols = real_estate['Symbol'].tolist()

real_estate_symbols


['PLD',
 'AMT',
 'EQIX',
 'SPG',
 'WELL',
 'PSA',
 'CCI',
 'DLR',
 'O',
 'CSGP',
 'EXR',
 'VICI',
 'CBRE',
 'AVB',
 'WY',
 'SBAC',
 'EQR',
 'IRM',
 'ARE',
 'KSPI',
 'INVH',
 'BEKE',
 'VTR',
 'SUI',
 'ESS',
 'MAA',
 'AMH',
 'HST',
 'UDR',
 'KIM',
 'ELS',
 'GLPI',
 'WPC',
 'LAMR',
 'DOC',
 'REXR',
 'REG',
 'CPT',
 'CUBE',
 'BXP',
 'NLY',
 'JLL',
 'EGP',
 'FRT',
 'OHI',
 'FSV',
 'NNN',
 'COLD',
 'FR',
 'STAG',
 'RHP',
 'AGNC',
 'BRX',
 'STWD',
 'CIGI',
 'ADC',
 'TRNO',
 'VNO',
 'RITM',
 'RYN',
 'HR',
 'NSA',
 'PECO',
 'KRG',
 'AIRC',
 'EPRT',
 'KRC',
 'APLE',
 'HHH',
 'RLJ',
 'MAC',
 'IRT',
 'PCH',
 'PK',
 'CUZ',
 'BXMT',
 'SLG',
 'SBRA',
 'CTRE',
 'SKT',
 'JOE',
 'EPR',
 'TCN',
 'DBRG',
 'SITC',
 'HASI',
 'CDP',
 'BNL',
 'ESRT',
 'IIPR',
 'DEI',
 'LXP',
 'MPW',
 'OUT',
 'HIW',
 'VTMX',
 'NMRK',
 'NHI',
 'ESBA',
 'ABR',
 'SHO',
 'CWK',
 'FCPT',
 'UE',
 'EQC',
 'DRH',
 'PEB',
 'AKR',
 'FOR',
 'OPEN',
 'EXPI',
 'GNL',
 'IVT',
 'CXW',
 'ROIC',
 'AAT',
 'XHR',
 'ARI',
 'NXRT',
 'RC',
 'JBGS',

In [ ]:
# Real-Estate sektörü için veri indirme ve özellik çıkarımı
real_estate_data = get_sector_data(real_estate_symbols, 'Real-Estate')

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

In [ ]:
energy = pd.read_csv('../data/stock_sectors/energy.csv')

energy_symbols = energy['Symbol'].tolist()

energy_data = get_sector_data(energy_symbols, 'Energy')

In [ ]:
# Finance, Technology ve Healthcare sektörlerinden elde edilen özelliklerin listesi
selected_features = relevant_features.columns.tolist()

def find_similar_sector(sector_data):

    if 'Date' not in sector_data.columns:
        sector_data.reset_index(inplace=True)
        sector_data.rename(columns={'index': 'Date'}, inplace=True)


    # Tarih sütununu datetime türüne dönüştürme
    sector_data['Date'] = pd.to_datetime(sector_data['Date'])

    # Tsfresh ile özellik çıkarımı
    sector_features = extract_features(sector_data, column_id='Symbol', column_sort='Date', column_value='Monthly Return',
                                            default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())

    # Impute ve özellikleri ölçeklendirme
    impute(sector_features)

    # Seçilen özellikleri kullanarak sektör verisini filtreleyin
    sector_selected_features = sector_features[selected_features]

    # Modeli kullanarak Real-Estate verisi üzerinde tahmin yapma
    sector_predictions = model.predict(sector_selected_features)

    # Tahmin sonuçlarını işleme ve analiz etme
    sector_predictions_mapped = le.inverse_transform(sector_predictions)
    predictions_count = pd.Series(sector_predictions_mapped).value_counts()
    majority_sector = predictions_count.idxmax()
    print(f" {sector_data['Sector']} sektöründeki hisse senetleri çoğunlukla {majority_sector} sektörüne benziyor.")

In [ ]:
find_similar_sector(real_estate_data)

,Monthly Return,Sector,Symbol
Date,,,
2005-04-30,1.177969,Energy,XOM
2005-05-31,1.096576,Energy,XOM
2005-06-30,0.896459,Energy,XOM
2005-07-31,0.960899,Energy,XOM
2005-08-31,1.039123,Energy,XOM
...,...,...,...
2023-11-30,0.617284,Energy,TPET
2023-12-31,0.513804,Energy,TPET
2024-01-31,0.510000,Energy,TPET


In [ ]:
find_similar_sector(energy_data)

Feature Extraction: 100%|███████████████████████| 20/20 [00:04<00:00,  4.58it/s]
/Users/ahmetbarisdurak/anaconda3/lib/python3.11/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['Monthly Return__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


 0        Energy
1        Energy
2        Energy
3        Energy
4        Energy
          ...  
41470    Energy
41471    Energy
41472    Energy
41473    Energy
41474    Energy
Name: Sector, Length: 41475, dtype: object sektöründeki hisse senetleri çoğunlukla Healthcare sektörüne benziyor.
